In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Make sure the path corresponds to where main.py script saves the file
data_path = '../temp/search_results.csv'
data = pd.read_csv(data_path)

# Display the first few rows of the data to verify it loaded correctly
print(data.head())

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(pd.to_datetime(data['Time Period Start Date']), data['Value'], marker='o')
plt.title('Data Visualized Over Time')
plt.xlabel('Time Period Start Date')
plt.ylabel('Value')
plt.grid(True)
plt.show()